# MoI ML CNN Experiment

- Train op fashion mnist
- Laat experimenten zien: kies 2 - 3 params, kies een range en voer tests uit. Doel is 93% accuracy.

In [6]:
import torch
import torch.nn as nn
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../")

In [9]:
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin


In [14]:
# Load config for this notebook
gin.parse_config_file("cnn.gin")

ParsedConfigFileIncludesAndImports(filename='cnn.gin', imports=[], includes=[])

In [15]:
# Get MNIST data
train_dataloader, test_dataloader = make_dataset.get_MNIST()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/raw/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/raw/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/raw/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/raw/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/raw/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/raw/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/raw/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/raw/FashionMNIST/raw



In [12]:
# Now do an experiment. First create a CNN class that can have different convolutions
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Set some other parameters
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
from src.models import train_model

# Define model
class ExpCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutions are set separately
       
        # Dense is default
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def set_convolution(self, conv : nn.Sequential):
        self.convolutions = conv

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

# Loop through some kernel sizes
for ksize in range(2,4):
    conv = nn.Conv2d(
        in_channels=1, 
        out_channels=32,
        kernel_size=ksize,
        padding=(1,1),
        stride=2)
    
    model = ExpCNN().to(device)
    model.set_convolution(nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=ksize, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            # The original:
            # nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),
            # nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2),


        ))

    # Now train
    model = train_model.trainloop(
        epochs=2, # was 10
        model=model,
        optimizer=optimizer,
        learning_rate=1e-3,
        loss_fn=loss_fn,
        metrics=[accuracy],
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        log_dir="../models/test/",
        train_steps=len(train_dataloader),
        eval_steps=len(test_dataloader),
    )


2022-11-30 15:00:29.237 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to models/test/20221130-1500


Using cpu device


100%|██████████| 1875/1875 [00:52<00:00, 35.59it/s]
2022-11-30 15:01:26.813 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.6524 test 0.5236 metric ['0.7986']
100%|██████████| 1875/1875 [00:50<00:00, 37.03it/s]
2022-11-30 15:02:20.428 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.4138 test 0.3994 metric ['0.8540']
100%|██████████| 2/2 [01:49<00:00, 54.89s/it]
2022-11-30 15:02:20.439 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to models/test/20221130-1502
100%|██████████| 1875/1875 [00:49<00:00, 37.98it/s]
2022-11-30 15:03:13.170 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.6511 test 0.4833 metric ['0.8192']
100%|██████████| 1875/1875 [00:50<00:00, 37.25it/s]
2022-11-30 15:04:06.576 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.4119 test 0.3905 metric ['0.8619']
100%|██████████| 2/2 [01:45<00:00, 52.98s/it]


Checkout results using tensorboard: 
- run `tensorboard --logdir models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`